## Google Photos API - Download images from Google Photos using Python

Using the Google Photos REST API you can download, upload and modify images stored in Google Photos.

The following steps describe how to set up a simple project that lets you use Python to download images from Google Photos:

## Create virtualenv and install required packages

1. Open the terminal and navigate to your working directory. The folder structure of the repo includes the following directories:

    * **credentials**: folder to store the credentials you need to authenticate your "Python App" to the Google Photos Library
    * **media_items_list**: every time the script runs, I want to save a .csv file with all Google Photos media items and the corresponding metadata uploaded in the defined time period
    * **downloads**: storing downloaded images from Google Photos


2. Create a virtual environment `python3 -m venv venv`, activate it `. ./venv/bin/activate` and install requirements `pip install -r requirements.txt`

3. Install ipykernel which provides the IPython kernel for Jupyter: `pip install ipykernel` and add your virtual environment to Jupyter: `python -m ipykernel install --user --name=venv` 

    You can check the installation by navigating to /Users/<user>/Library/Jupyter/kernels. There should be a new directory called 'venv'. In the folder you can find the file 'kernel.json', which contains the path for the used python installation is defined.

4. Start jupyter notebook or jupyter lab: `jupyter lab .` and select the just created environment "venv" as Kernel

![](read_me_img/select_kernel.png)

## Enable Google API

5. Enable Google Photos API Service

   1. Go to the Google API Console [https://console.cloud.google.com/](https://console.cloud.google.com/). 
   2. From the menu bar, select a project or create a new project.
   
      ![](read_me_img/gifs/create_new_project_speed.gif)
      
   3. To open the Google API Library, from the Navigation menu, select APIs & Services > Library. 
   4. Search for "Google Photos Library API". Select the correct result and click "enable". If its already enabled, click "manage"
   
       ![](read_me_img/gifs/enable_api_speed.gif)
       
   5. Afterwards it will forward you to the "Photos API/Service details" page (https://console.cloud.google.com/apis/credentials)


6. Configure "OAuth consent screen" ([Source](https://stackoverflow.com/questions/65184355/error-403-access-denied-from-google-authentication-web-api-despite-google-acc))

   1. Go back to the Photos API Service details page and click on "[OAuth consent screen](https://console.cloud.google.com/apis/credentials/consent)" on the left side (below "Credentials") 
   2. Add a Test user: Use the email of the account you want to use for testing the API call
   
        ![](read_me_img/add_test_user.png)

7. Create API/OAuth credentials

   1. On the left side at the Google Photos API Service page click on Credentials
   2. Click on "Create Credentials" and create a OAuth client ID
   3. As application type I am choosing "Desktop app" and give your client you want to use to call the API a name
   4. Download the JSON file to the created credentials, rename it to "client_secret.json" and save it in the folder "credentials"
   
        ![](read_me_img/gifs/create_credentials_speed.gif)

## Install and import required packages

In [1]:
%%capture capt 
#saves the output to variable capt, to print output capt.stdout, capt.stderr
!pip install -r "requirements.txt"
!pip freeze > requirements.txt

In [2]:
!which python
!which pip

/Users/dominikpolzer/Documents/repos/repos_test/google-photos-api/venv/bin/python
/Users/dominikpolzer/Documents/repos/repos_test/google-photos-api/venv/bin/pip


## Use the Google Photo Library API for the first time:

The following section shows how to use OAuth Credentials for authentication with the Google Library API. The code section below covers the following steps:

8. Create a service for the first time:

    1. Initialize GooglePhotosApi `google_photos_api = GooglePhotosApi()`

    2. Create Service using the `client_secret.json` file: `service = google_photos_api.create_service()`
        
        
       <b>Calling the API for the first time:</b>
       1. Google will ask you if you want to grant the App the required permissions you defined with the scope:
       ![](read_me_img/sign_in_google_acc.png)
       2. Since its just a test app at the moment, Google will make you aware of that > Click on "Continue"
       3. Once you granted the app the required permissions, you will see a "token_......pickle" file created in the folder "credentials". This token file will be used for future calls.

In [3]:
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
#from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request
import requests

class GooglePhotosApi:
    def __init__(self,
                 api_name = 'photoslibrary',
                 client_secret_file= r'./credentials/client_secret.json',
                 api_version = 'v1',
                 scopes = ['https://www.googleapis.com/auth/photoslibrary']):
        '''
        Args:
            client_secret_file: string, location where the requested credentials are saved
            api_version: string, the version of the service
            api_name: string, name of the api e.g."docs","photoslibrary",...
            api_version: version of the api

        Return:
            service:
        '''

        self.api_name = api_name
        self.client_secret_file = client_secret_file
        self.api_version = api_version
        self.scopes = scopes
        self.cred_pickle_file = f'./credentials/token_{self.api_name}_{self.api_version}.pickle'

        self.cred = None

    def run_local_server(self):
        # is checking if there is already a pickle file with relevant credentials
        if os.path.exists(self.cred_pickle_file):
            with open(self.cred_pickle_file, 'rb') as token:
                self.cred = pickle.load(token)

        # if there is no pickle file with stored credentials, create one using google_auth_oauthlib.flow
        if not self.cred or not self.cred.valid:
            if self.cred and self.cred.expired and self.cred.refresh_token:
                self.cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.client_secret_file, self.scopes)
                self.cred = flow.run_local_server()

            with open(self.cred_pickle_file, 'wb') as token:
                pickle.dump(self.cred, token)
        
        return self.cred


In [4]:
# initialize photos api and create service
google_photos_api = GooglePhotosApi()
creds = google_photos_api.run_local_server()

### Use pythons requests module and the token file to retrieve data from Google Photos

9. Use requests python module to send http requests to the Media Items API

    The following function sends a post request to the Media API to get a list of all entries. Since the API return is limited to 100 items, the search is narrowed down to one day. Thus, the call would only be a problem if more than 100 images were created/uploaded on one day.

In [5]:
import json
import requests

def get_response_from_medium_api(year, month, day):
    url = 'https://photoslibrary.googleapis.com/v1/mediaItems:search'
    payload = {
                  "filters": {
                    "dateFilter": {
                      "dates": [
                        {
                          "day": day,
                          "month": month,
                          "year": year
                        }
                      ]
                    }
                  }
                }
    headers = {
        'content-type': 'application/json',
        'Authorization': 'Bearer {}'.format(creds.token)
    }
    
    try:
        res = requests.request("POST", url, data=json.dumps(payload), headers=headers)
    except:
        print('Request error') 
    
    return(res)

Use the response of the API to write the results and required metadata into a data frame:

In [6]:
def list_of_media_items(year, month, day, media_items_df):
    '''
    Args:
        year, month, day: day for the filter of the API call 
        media_items_df: existing data frame with all find media items so far
    Return:
        media_items_df: media items data frame extended by the articles found for the specified tag
        items_df: media items uploaded on specified date
    '''

    items_list_df = pd.DataFrame()
    
    # create request for specified date
    response = get_response_from_medium_api(year, month, day)

    try:
        for item in response.json()['mediaItems']:
            items_df = pd.DataFrame(item)
            items_df = items_df.rename(columns={"mediaMetadata": "creationTime"})
            items_df.set_index('creationTime')
            items_df = items_df[items_df.index == 'creationTime']

            #append the existing media_items data frame
            items_list_df = pd.concat([items_list_df, items_df])
            media_items_df = pd.concat([media_items_df, items_df])
    
    except:
        print(response.text)

    return(items_list_df, media_items_df)

## Use the defined functions to download media items from Google Photos

1. Create a list with all files already downloaded to the /downloads/ folder
2. Define a list of all dates from start date to end date (today)
3. Execute the API call for all dates to get a list with all media items. API returns:
    * **id**
    * **filename**
    * **baseUrl**: Base URLs within the Google Photos Library API allow you to access the bytes of the media items. They are valid for 60 minutes. (https://developers.google.com/photos/library/guides/access-media-items)


4. Compare list of media items with files downloaded in /downloads/ with media items in Google Photos, to download items which are not downloaded yet. You can now use the baseUrl and the python requests module to send a get request for each media item.
5. Save a list as with all media items as .csv in /media_items_list/

In [8]:
import pandas as pd
from datetime import date, timedelta, datetime
import requests

# Images should only be downloaded if they are not already available in downloads
# Herefor the following code snippet, creates a list with all filenames in the /downloads/ folder
files_list = os.listdir(r'./downloads')
files_list_df = pd.DataFrame(files_list)
files_list_df = files_list_df.rename(columns={0: "filename"})
files_list_df.head(2)

# create a list with all dates between start date and today
sdate = date(2021,4,16)   # start date
edate = date.today()
date_list = pandas.date_range(sdate,edate-timedelta(days=1),freq='d')
print(date_list)

media_items_df = pd.DataFrame()

for date in date_list:
    
    # get a list with all media items for specified date (year, month, day)
    items_df, media_items_df = list_of_media_items(year = date.year, month = date.month, day = date.day, media_items_df = media_items_df)

    if len(items_df) > 0:
        # full outer join of items_df and files_list_df, the result is a list of items of the given 
        #day that have not been downloaded yet
        items_not_yet_downloaded_df = pd.merge(items_df, files_list_df,on='filename',how='left')
        items_not_yet_downloaded_df.head(2)

        # download all items in items_not_yet_downloaded
        for index, item in items_not_yet_downloaded_df.iterrows():
            url = item.baseUrl
            response = requests.get(url)

            file_name = item.filename
            destination_folder = './downloads/'

            with open(os.path.join(destination_folder, file_name), 'wb') as f:
                f.write(response.content)
                f.close()
                
        print(f'Downloaded items found for date: {date.year} / {date.month} / {date.day}')
    else:
        print(f'No media items found for date: {date.year} / {date.month} / {date.day}')
            
#save a list of all media items to a csv file
current_datetime = str(datetime.now())
filename = f'item-list-{current_datetime}.csv'

#save a list with all items in specified time frame
media_items_df.to_csv(f'./media_items_list/{filename}', index=True)

DatetimeIndex(['2021-04-16', '2021-04-17', '2021-04-18', '2021-04-19',
               '2021-04-20', '2021-04-21', '2021-04-22', '2021-04-23',
               '2021-04-24', '2021-04-25',
               ...
               '2022-05-02', '2022-05-03', '2022-05-04', '2022-05-05',
               '2022-05-06', '2022-05-07', '2022-05-08', '2022-05-09',
               '2022-05-10', '2022-05-11'],
              dtype='datetime64[ns]', length=391, freq='D')
{}

No media items found for date: 2021 / 4 / 16
Downloaded items found for date: 2021 / 4 / 17
Downloaded items found for date: 2021 / 4 / 18
Downloaded items found for date: 2021 / 4 / 19
{}

No media items found for date: 2021 / 4 / 20
Downloaded items found for date: 2021 / 4 / 21
Downloaded items found for date: 2021 / 4 / 22
Downloaded items found for date: 2021 / 4 / 23
Downloaded items found for date: 2021 / 4 / 24
{}

No media items found for date: 2021 / 4 / 25
Downloaded items found for date: 2021 / 4 / 26
Downloaded items found for 

In [9]:
!git add .
!git commit -m "update jupyter notebook"
!git push

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'dominikpolzer@Dominiks-Air.(none)')
Everything up-to-date
